# Transfer Learning

In [ ]:
%pylab inline

In [ ]:
import re

from bigdl.nn.criterion import CrossEntropyCriterion
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType

from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Dense, Input, Flatten
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *
import numpy as np
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
from glob import glob



In [ ]:
from bigdl.nn.criterion import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType, ArrayType
from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Input, Flatten, Dense
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *
from zoo.feature.image.imagePreprocessing import *

import random
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from zoo.pipeline.api.keras.metrics import AUC
from pyspark.sql.types import *

import numpy as np
from sklearn.metrics import roc_auc_score
from pyspark.ml import Pipeline
from bigdl.optim.optimizer import * 

In [ ]:
sparkConf = SparkConf().setAppName("testMutipleLabels")
sc = init_nncontext(sparkConf)

In [ ]:
sparkConf = create_spark_conf().setAppName("testMutipleLabels")
sc = init_nncontext(sparkConf)
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [ ]:
path="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/red_shirt/"
print path
new_path="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/"
print new_path

In [ ]:
type(sc)

In [ ]:
from bigdl.util.common import *
from bigdl.transform.vision.image import *
import matplotlib.pyplot as plt

manually set model_path and image_path for training

1. model_path = path to the pre-trained models. (E.g. path/to/model/bigdl_inception-v1_imagenet_0.4.0.model)

2. image_path = path to the folder of the training images. (E.g. path/to/data/dogs-vs-cats/demo/\*/\*)

In [ ]:
model_path = "/home/mahmood/analytics-zoo/demo/bigdl_inception-v1_imagenet_0.4.0.model"

#image_path = "file:///home/mahmood/analytics-zoo/demo/*/*"
#image_path=ImageFrame.read("hdfs:///datasets/xray/train/*", sc, 2) this does not work 
#image_path=ImageFrame.read("hdfs://hinfra/datasets/xray/train/*",sc,2)
#image_path = "file:///home/mahmood/Downloads/*/*"
#image_path = "file:///home/mahmood/analytics-zoo/demo/*/*"
image_path = "file:///home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/"
imageDF = NNImageReader.readImages(image_path, sc)

In [ ]:
image_path

In [ ]:
from glob import glob

txtfiles =[]
Image_index=[]
for file in glob("/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/*"):
    txtfiles.append(file.split('/'))
for i in range(len(txtfiles)):  
    Image_index.append(txtfiles[i][8:9])
    
print len(txtfiles)

In [ ]:
Labels1=[]
for j in range(len(Image_index)):
    s=re.sub('[^A-Za-z_.]', '', str(Image_index[j]))
    #print s
    s=s.strip('_.jpg')
    s=s.strip('_.jpeg')
    s=s.strip('_.JPG')
    s=s.strip('_.pn')
    Labels1.append(s)
Labels1[0:10]
    

In [ ]:
type(s)

In [ ]:
label=Image_index

In [ ]:
Img_indx=[]
for j in range(len(Image_index)):
    s=re.sub('[^A-Za-z0-9_.]', '', str(Image_index[j]))
    #print s
    
    Img_indx.append(s)
Img_indx[0:10]

In [ ]:
import csv
from itertools import izip

with open('AllLabels.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Img_indx,Labels1))

In [ ]:

 
# open file
with open('AllLabels.csv', 'rb') as f:
    reader = csv.reader(f)
 
    # read file row by row
    for row in reader:
        print row

In [ ]:
image_path='/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/'
label_path = '/home/mahmood/analytics-zoo/apps/dogs-vs-cats/AllLabels.csv'

In [ ]:
getLabel = udf(lambda x: text_to_label(x), DoubleType())

In [ ]:
import pandas as pd
labelDF=pd.read_csv(label_path)

In [ ]:
labelDF.head()

In [ ]:
sqlCtx = SQLContext(sc)
labelDFsp = sqlCtx.createDataFrame(labelDF)

In [ ]:
label_texts = list("""black,jeans,blue,dress,shirt,red""".replace("\n", "").split(","))
print label_texts

label_map = {k: v for v, k in enumerate(label_texts)}
print label_map

In [ ]:
def text_to_label(text):
    arr = [0.0] * len(label_texts)
    for l in text.split("_"):
        arr[label_map[l]] = 1.0
     
    return arr
print text_to_label("black_jeans")   

In [ ]:
getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))

In [ ]:
labelDF11 = labelDFsp.select("Image_index", "labels") \
   .withColumn("label", getLabel(col('labels')))



In [ ]:
labelDF11.select("Image_index","label").show()

In [ ]:
#getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))
getName = udf(lambda row: os.path.basename(row[0]), StringType())
imageDF = NNImageReader.readImages(image_path, sc, resizeH=300, resizeW=300 ) \
    .withColumn("Image_index", getName(col('image')))


labelDF = spark.read.load(label_path, format="csv", sep=",", inferSchema="true", header="true") \
   .select("Image Index", "Finding Labels") \
   .withColumn("label", getLabel(col('Finding Labels')))

In [ ]:
imageDF.head(1)

# MY code start from here

label_texts = list("""Atelectasis, Consolidation, Infiltration, Pneumothorax, Edema, Emphysema, Fibrosis, Effusion, Pneumonia, Pleural_Thickening, Cardiomegaly, Nodule, Mass, Hernia, No Finding""".replace("\n", "").split(", "))
label_map = {k: v for v, k in enumerate(label_texts)}

#-------------------------------------------
def text_to_label(text):
    arr = [0.0] * len(label_texts)
    
    for l in text.split(","):
        arr[label_map[l]] = 1.0
     
    return arr
#------------------------------------------
getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))
getName = udf(lambda row: os.path.basename(row[0]), StringType())
imageDF = NNImageReader.readImages(image_path, sc, resizeH=256, resizeW=256) \
    .withColumn("Image Index", getName(col('image')))
labelDF = spark.read.load(label_path, format="csv", sep=",", inferSchema="true", header="true") \
   .select("Image Index", "Finding Labels") \
   .withColumn("label", getLabel(col('Finding Labels')))

In [ ]:
train_df = imageDF.join(labelDF11, on="Image_index", how="inner")
(trainingDF, validationDF) = train_df.randomSplit([0.7, 0.3])

In [ ]:
(trainingDF, validationDF) = labelDF.randomSplit([0.9, 0.1])
labelDF.select("name","label").show(20)

In [ ]:

type(trainingDF)


## Fine-tune a pre-trained model

We fine-tune a pre-trained model by removing the last few layers, freezing the first few layers, and adding some new layers.

In [ ]:
transformer = ChainedPreprocessing(
        [RowToImageFeature(), ImageResize(256, 256), ImageCenterCrop(224, 224),
         ImageChannelNormalize(123.0, 117.0, 104.0), ImageMatToTensor(), ImageFeatureToTensor()])

### Load a pre-trained model

We use the Net API to load a pre-trained model, including models saved by Analytics Zoo, BigDL, Torch, Caffe and Tensorflow. Please refer to [Net API Guide](https://analytics-zoo.github.io/master/#APIGuide/PipelineAPI/net/).

In [ ]:
full_model = Net.load_bigdl(model_path)

### Remove the last few layers

Here we print all the model layers and you can choose which layer(s) to remove.

When a model is loaded using Net, we can use the newGraph(output) api to define a Model with the output specified by the parameter. 

In [ ]:
for layer in full_model.layers:
    print (layer.name())
model = full_model.new_graph(["pool5/drop_7x7_s1"])

The returning model's output layer is "pool5/drop_7x7_s1".

### Freeze some layers

We freeze layers from input to pool4/3x3_s2 inclusive.

model.freeze_up_to(["pool4/3x3_s2"])

In [ ]:
inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(6, activation="sigmoid")(flatten)
lrModel = Model(inputNode, logits)

### Add a few new layers

In [ ]:
val_summary = ValidationSummary(log_dir="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/logDirectroy/log", app_name="testMutipleLabels")

inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(2)(flatten)
lrModel = Model(inputNode, logits)
classifier = NNClassifier(lrModel, CrossEntropyCriterion(), transformer) \
        .setLearningRate(0.003).setBatchSize(24).setMaxEpoch(1).setFeaturesCol("image") \
        .setCachingSample(False)
pipeline = Pipeline(stages=[classifier])


In [ ]:
classifier = NNEstimator(lrModel, MultiLabelSoftMarginCriterion(), transformer, SeqToTensor([6])) \
     .setLearningRate(0.001).setBatchSize(24).setMaxEpoch(10).setFeaturesCol("image")\
     .setCachingSample(False)\
     .setValidation(EveryEpoch(), validationDF,[AUC()],  24)
#\
 #    .setValidationSummary(val_summary)
#\
    # .setCheckpoint("/home/mahmood/analytics-zoo/apps/dogs-vs-cats/logDirectroy/checkpoint", EveryEpoch(),False)

In [ ]:
nnModel = classifier.fit(trainingDF)
print("Finished training")

In [ ]:
nnModel.transform(trainingDF).show(5)

predictionDF = nnModel.transform(validationDF).cache()

predictionDF.select("Image_index","label","prediction").show(5)